# Logistic Regression VS Max Entropy 和 Theano实现

## 1.动机
首先，来说说为什么要写LR和ME。首先，

In [1]:
import numpy
import theano
import theano.tensor as T
rng = numpy.random

N = 400                                  # training sample size
feats = 784                               # number of input variables

# generate a dataset: D = (input_values, target_class)
D = (rng.randn(N, feats), rng.randint(size=N, low=0, high=2))
training_steps = 10000

# Declare Theano symbolic variables
x = T.dmatrix("x")
y = T.dvector("y")

# initialize the weight vector w randomly
#
# this and the following bias variable b
# are shared so they keep their values
# between training iterations (updates)
w = theano.shared(rng.randn(feats), name="w")

# initialize the bias term
b = theano.shared(0., name="b")

print("Initial model:")
print(w.get_value())
print(b.get_value())

# Construct Theano expression graph
p_1 = 1 / (1 + T.exp(-T.dot(x, w) - b))   # Probability that target = 1
prediction = p_1 > 0.5                    # The prediction thresholded
xent = -y * T.log(p_1) - (1-y) * T.log(1-p_1) # Cross-entropy loss function
cost = xent.mean() + 0.01 * (w ** 2).sum()# The cost to minimize
gw, gb = T.grad(cost, [w, b])             # Compute the gradient of the cost
                                          # w.r.t weight vector w and
                                          # bias term b
                                          # (we shall return to this in a
                                          # following section of this tutorial)

# Compile
train = theano.function(
          inputs=[x,y],
          outputs=[prediction, xent],
          updates=((w, w - 0.1 * gw), (b, b - 0.1 * gb)))
predict = theano.function(inputs=[x], outputs=prediction)

# Train
for i in range(training_steps):
    pred, err = train(D[0], D[1])

print("Final model:")
print(w.get_value())
print(b.get_value())
print("target values for D:")
print(D[1])
print("prediction on D:")
print(predict(D[0]))

Using gpu device 0: GeForce GTX 960M (CNMeM is disabled, cuDNN not available)


Initial model:
[  1.05609400e-01  -4.24518336e-01   7.34994098e-01   1.26457149e+00
  -2.50289124e-01  -1.20086246e+00  -1.52054267e+00  -2.08441284e-02
  -1.99477984e+00   1.13171994e+00  -1.00068171e+00   3.01789042e-02
   1.75514447e+00  -8.63187907e-01  -1.11891006e-01   2.27729508e-01
  -9.62856484e-01  -6.24412011e-01  -2.59886423e-01   7.68623488e-01
  -6.15863405e-01   1.32709445e+00  -2.17738665e+00  -1.10490083e+00
   8.20437286e-01  -1.89606077e+00  -3.54358628e-01   1.24363143e+00
  -8.54204552e-01   2.71741080e+00   4.01346024e-01   3.26753624e-01
   1.51347108e-01   2.99123140e-01   8.92115826e-01  -1.96788919e+00
   1.11157232e+00   8.74149261e-01   1.00128993e+00   1.32657962e-01
  -1.45997516e-01   5.13728491e-01   4.25392018e-01   2.09859381e-01
   8.67116707e-01   1.64389698e+00   4.45537150e-01   5.01740692e-01
  -1.38877362e+00  -9.64916899e-01  -3.93130963e-03   2.61953300e-01
   2.11722463e-01   2.82392423e-01   9.21789854e-01   2.94003978e-01
   1.97101572e-01  